In [2]:
print("project setup successful")

project setup successful


In [3]:
import os

In [4]:
import osos.listdir("data")


SyntaxError: invalid syntax (1717094852.py, line 1)

In [5]:
import os
os.listdir("data")

['Transactions.zip']

In [6]:
import os
os.listdir()

['.ipynb_checkpoints', '01_file_index.ipynb', 'data', 'output', 'src']

In [7]:
os.getcwd()

'C:\\Users\\ftorr\\FPA_PROJECT'

In [11]:
import zipfile
zip_path = "data/Transactions.zip"
extract_path = "data/Transactions"
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction completed.")

Extraction completed.


In [12]:
os.listdir("data")

['Transactions', 'Transactions.zip']

In [13]:
os.listdir("data/Transactions")

['2023 Budget FX Rates.xlsx', 'Transacciones']

In [14]:
os.listdir("data/Transactions/Transacciones")

['2024']

In [15]:
os.listdir("data/Transactions/Transacciones/2024")

['01 January',
 '02 February',
 '03 March',
 '04 April',
 '05 May',
 '06 June',
 '07 July',
 '08 August',
 '09 September',
 '10 October',
 '11 November',
 '12 December']

In [16]:
os.listdir("data/Transactions/Transacciones/2024/01 January")

['04-01-24', '11-01-24', '26-01-24']

In [17]:
os.listdir("data/Transactions/Transacciones/2024/01 January/04-01-24")

['DataTrans_0.csv', 'DataTrans_1.xlsx']

In [3]:
import os
import re
import pandas as pd
from datetime import datetime

def build_file_index(base_dir: str) -> pd.DataFrame:
    """
    Scans base_dir recursively and returns a table of all transaction files.
    Works for any year (2024, 2025, etc.) as long as folders follow:
    Transacciones/<YEAR>/<MONTH>/<DD-MM-YY>/DataTrans_*.csv|.xlsx
    """
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()

            # Only transaction files
            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)

            # Try to parse metadata from the path
                        parts_norm = os.path.normpath(filepath).split(os.sep)
            parts_lower = [p.lower() for p in parts_norm]
            
            year = None
            month_folder = None
            statement_folder = None
            statement_date = None
            
            if "transacciones" in parts_lower:
                i = parts_lower.index("transacciones")
            
                if i + 3 < len(parts_norm):
                    year_str = parts_norm[i+1]
                    month_folder = parts_norm[i+2]
                    statement_folder = parts_norm[i+3]
            
                    if year_str.isdigit():
                        year = int(year_str)
            
                    m = re.match(r"(\d{2})-(\d{2})-(\d{2})$", statement_folder)
                    if m:
                        dd, mm, yy = map(int, m.groups())
                        statement_date = datetime(2000 + yy, mm, dd).date()

            rows.append({
                "filepath": filepath,
                "filetype": os.path.splitext(filepath)[1].lower().replace(".", ""),
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)

    # Derived fields
    df["statement_date"] = pd.to_datetime(df["statement_date"])
    df["month"] = df["statement_date"].dt.month
    df["quarter"] = df["statement_date"].dt.quarter

    # Sort + unique ID per file
    df = df.sort_values(["statement_date", "filepath"]).reset_index(drop=True)
    df["source_id"] = (
        df["statement_date"].dt.strftime("%Y-%m-%d")
        + "_"
        + df.groupby("statement_date").cumcount().astype(str).str.zfill(3)
    )

    return df


In [5]:
base_dir = "data/Transactions/Transacciones"
file_index = build_file_index(base_dir)

print("Files found:", len(file_index))
print(file_index["filetype"].value_counts())
print(file_index["statement_date"].min(), "→", file_index["statement_date"].max())
file_index.head()

Files found: 537
filetype
xlsx    303
csv     234
Name: count, dtype: int64
NaT → NaT


,filepath,filetype,year,month_folder,statement_folder,statement_date,month,quarter,source_id
0,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN
1,data/Transactions/Transacciones\2024\01 Januar...,xlsx,None,None,None,NaT,NaN,NaN,NaN
2,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN
3,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN
4,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN


In [6]:
file_index = build_file_index(base_dir)

print("Files found:", len(file_index))
print(file_index["filetype"].value_counts())
print(file_index["statement_date"].min(), "→", file_index["statement_date"].max())
file_index.head()

Files found: 537
filetype
xlsx    303
csv     234
Name: count, dtype: int64
NaT → NaT


,filepath,filetype,year,month_folder,statement_folder,statement_date,month,quarter,source_id
0,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN
1,data/Transactions/Transacciones\2024\01 Januar...,xlsx,None,None,None,NaT,NaN,NaN,NaN
2,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN
3,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN
4,data/Transactions/Transacciones\2024\01 Januar...,csv,None,None,None,NaT,NaN,NaN,NaN


In [7]:
import os

sample = file_index.loc[0, "filepath"]
print("SAMPLE PATH:\n", sample)

parts = os.path.normpath(sample).split(os.sep)
print("\nTOKENS:")
for p in parts:
    print(p)

SAMPLE PATH:
 data/Transactions/Transacciones\2024\01 January\04-01-24\DataTrans_0.csv

TOKENS:
data
Transactions
Transacciones
2024
01 January
04-01-24
DataTrans_0.csv


In [8]:
file_index = build_file_index(base_dir)

print(file_index[["year","month_folder","statement_folder","statement_date"]].head(10))
print(file_index["statement_date"].min(), "→", file_index["statement_date"].max())


   year month_folder statement_folder statement_date
0  None         None             None            NaT
1  None         None             None            NaT
2  None         None             None            NaT
3  None         None             None            NaT
4  None         None             None            NaT
5  None         None             None            NaT
6  None         None             None            NaT
7  None         None             None            NaT
8  None         None             None            NaT
9  None         None             None            NaT
NaT → NaT


In [9]:
import os
import re
import pandas as pd
from datetime import datetime

YEAR_RE = re.compile(r"(20\d{2})")                 # 2020-2099
DATE_RE = re.compile(r"(\d{2}-\d{2}-\d{2})")       # dd-mm-yy

def build_file_index(base_dir: str) -> pd.DataFrame:
    rows = []

    for root, _, files in os.walk(base_dir):
        for f in files:
            f_low = f.lower()
            if "datatrans" not in f_low:
                continue
            if not f_low.endswith((".csv", ".xlsx")):
                continue

            filepath = os.path.join(root, f)

            # Normalize path string so regex works consistently on Windows
            norm = os.path.normpath(filepath)

            # Find year anywhere in the path
            year_match = YEAR_RE.search(norm)
            year = int(year_match.group(1)) if year_match else None

            # Find the date folder anywhere in the path (take the LAST match in case)
            date_matches = DATE_RE.findall(norm)
            statement_date = None
            statement_folder = None
            if date_matches:
                statement_folder = date_matches[-1]  # last one is safest
                dd, mm, yy = map(int, statement_folder.split("-"))
                statement_date = datetime(2000 + yy, mm, dd).date()

            # Try to infer "month_folder" (the folder right before the date folder)
            month_folder = None
            if statement_folder:
                parts = norm.split(os.sep)
                if statement_folder in parts:
                    j = parts.index(statement_folder)
                    if j - 1 >= 0:
                        month_folder = parts[j - 1]

            rows.append({
                "filepath": filepath,
                "filetype": os.path.splitext(filepath)[1].lower().replace(".", ""),
                "year": year,
                "month_folder": month_folder,
                "statement_folder": statement_folder,
                "statement_date": statement_date
            })

    df = pd.DataFrame(rows)

    # Derived fields
    df["statement_date"] = pd.to_datetime(df["statement_date"])
    df["month"] = df["statement_date"].dt.month
    df["quarter"] = df["statement_date"].dt.quarter

    # Sort + unique ID per file
    df = df.sort_values(["statement_date", "filepath"]).reset_index(drop=True)
    df["source_id"] = (
        df["statement_date"].dt.strftime("%Y-%m-%d")
        + "_"
        + df.groupby("statement_date").cumcount().astype(str).str.zfill(3)
    )

    return df


In [10]:
base_dir = "data/Transactions/Transacciones"
file_index = build_file_index(base_dir)

print("Files found:", len(file_index))
print(file_index["filetype"].value_counts())
print(file_index["statement_date"].min(), "→", file_index["statement_date"].max())
file_index[["year","month_folder","statement_folder","statement_date"]].head(10)

Files found: 537
filetype
xlsx    303
csv     234
Name: count, dtype: int64
2024-01-04 00:00:00 → 2024-12-29 00:00:00


,year,month_folder,statement_folder,statement_date
0,2024,01 January,04-01-24,2024-01-04
1,2024,01 January,04-01-24,2024-01-04
2,2024,01 January,11-01-24,2024-01-11
3,2024,01 January,11-01-24,2024-01-11
4,2024,01 January,11-01-24,2024-01-11
5,2024,01 January,11-01-24,2024-01-11
6,2024,01 January,26-01-24,2024-01-26
7,2024,01 January,26-01-24,2024-01-26
8,2024,01 January,26-01-24,2024-01-26
9,2024,01 January,26-01-24,2024-01-26


In [11]:
file_index.to_csv("output/file_index.csv", index=False)
print("Saved to output/file_index.csv")

Saved to output/file_index.csv


In [12]:
os.listdir("output")


['file_index.csv']

In [13]:
import pandas as pd

def read_one_transaction_file(filepath: str) -> pd.DataFrame:
    """
    Reads one DataTrans file (csv or xlsx) and returns a standardized dataframe.
    Expected columns: Client, Country, Currency, Transaction
    """
    ext = filepath.lower().split(".")[-1]

    if ext == "csv":
        df = pd.read_csv(filepath, sep=";")   # IMPORTANT: your CSV uses ;
    elif ext == "xlsx":
        df = pd.read_excel(filepath)
    else:
        raise ValueError(f"Unsupported file type: {filepath}")

    # Standardize column names (protect against minor variations)
    df.columns = [c.strip() for c in df.columns]

    # Keep only expected columns (will error early if file is malformed)
    expected = ["Client", "Country", "Currency", "Transaction"]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns {missing} in file: {filepath}")

    df = df[expected].copy()

    return df


In [14]:
sample_path = file_index.loc[0, "filepath"]
print(sample_path)

df_sample = read_one_transaction_file(sample_path)
df_sample.head()


data/Transactions/Transacciones\2024\01 January\04-01-24\DataTrans_0.csv


,Client,Country,Currency,Transaction
0,FLVWa,FIJI,MWK,1.626847e+06
1,FLVWa,GUYANA,LAK,4.454638e+07
2,FLVWa,BAHAMAS,MMK,1.758344e+06
3,EKKL3,BOTSWANA,TWD,3.409847e+04
4,EKKL3,OMAN,CLP,5.407556e+04


In [15]:
all_dfs = []

for _, row in file_index.iterrows():
    df = read_one_transaction_file(row["filepath"])

    # Attach metadata from file_index
    df["year"] = row["year"]
    df["statement_date"] = row["statement_date"]
    df["month"] = row["month"]
    df["quarter"] = row["quarter"]
    df["source_id"] = row["source_id"]

    all_dfs.append(df)

# Combine everything
transactions = pd.concat(all_dfs, ignore_index=True)

print("Total rows:", len(transactions))
transactions.head()

Total rows: 314754


,Client,Country,Currency,Transaction,year,statement_date,month,quarter,source_id
0,FLVWa,FIJI,MWK,1.626847e+06,2024,2024-01-04,1,1,2024-01-04_000
1,FLVWa,GUYANA,LAK,4.454638e+07,2024,2024-01-04,1,1,2024-01-04_000
2,FLVWa,BAHAMAS,MMK,1.758344e+06,2024,2024-01-04,1,1,2024-01-04_000
3,EKKL3,BOTSWANA,TWD,3.409847e+04,2024,2024-01-04,1,1,2024-01-04_000
4,EKKL3,OMAN,CLP,5.407556e+04,2024,2024-01-04,1,1,2024-01-04_000


In [16]:
fx_path = "data/Transactions/2023 Budget FX Rates.xlsx"

fx = pd.read_excel(fx_path)

print(fx.head())
print(fx.columns)


                             Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3
0                        MARSH MCLENNAN        NaN        NaN        NaN
1  FOREIGN EXCHANGE RATE SUMMARY REPORT        NaN        NaN        NaN
2                           2023 BUDGET        NaN        NaN        NaN
3                                   NaN        NaN        NaN        NaN
4                                   NaN        NaN       CURR     BUDGET
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')


In [17]:
fx_raw = pd.read_excel(fx_path, header=None)

fx_raw.head(15)


,0,1,2,3
0,NaN,NaN,NaN,NaN
1,MARSH MCLENNAN,NaN,NaN,NaN
2,FOREIGN EXCHANGE RATE SUMMARY REPORT,NaN,NaN,NaN
3,2023 BUDGET,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,CURR,BUDGET
6,COUNTRY,CURRENCY,CODE,RATE
7,ALBANIA,LEK,ALL,0.008751
8,ALGERIA,DINAR,DZD,0.00717
9,ANGOLA,KWANZA,AOA,0.001961


In [18]:
fx = pd.read_excel(fx_path, skiprows=6)

print(fx.head())
print(fx.columns)
print("Rows:", len(fx))


     COUNTRY CURRENCY CODE      RATE
0    ALBANIA      LEK  ALL  0.008751
1    ALGERIA    DINAR  DZD  0.007170
2     ANGOLA   KWANZA  AOA  0.001961
3  ARGENTINA     PESO  ARS  0.004545
4      ARUBA   FLORIN  AWG  0.558659
Index(['COUNTRY', 'CURRENCY', 'CODE', 'RATE'], dtype='object')
Rows: 137


In [19]:
fx_clean = fx[["CODE", "RATE"]].copy()

# Make CODE text clean
fx_clean["CODE"] = fx_clean["CODE"].astype(str).str.strip().str.upper()

# Ensure RATE is numeric
fx_clean["RATE"] = pd.to_numeric(fx_clean["RATE"], errors="coerce")

print(fx_clean.head())
print("Missing rates:", fx_clean["RATE"].isna().sum())


  CODE      RATE
0  ALL  0.008751
1  DZD  0.007170
2  AOA  0.001961
3  ARS  0.004545
4  AWG  0.558659
Missing rates: 0


In [20]:
transactions["Currency"] = transactions["Currency"].astype(str).str.strip().str.upper()

transactions_fx = transactions.merge(
    fx_clean,
    left_on="Currency",
    right_on="CODE",
    how="left"
)

transactions_fx["Transaction_USD"] = transactions_fx["Transaction"] * transactions_fx["RATE"]

print(transactions_fx[["Currency", "Transaction", "RATE", "Transaction_USD"]].head())
print("Currencies missing FX:", transactions_fx["RATE"].isna().sum())


  Currency   Transaction      RATE  Transaction_USD
0      MWK  1.626847e+06  0.000974      1585.232614
1      LAK  4.454638e+07  0.000058      2603.290197
2      MMK  1.758344e+06  0.000476       837.305845
3      TWD  3.409847e+04  0.032055      1093.039797
4      CLP  5.407556e+04  0.001053        56.921554
Currencies missing FX: 0


In [21]:
missing = transactions_fx["RATE"].isna().sum()
print("Missing FX rates:", missing)

unique_missing = transactions_fx.loc[
    transactions_fx["RATE"].isna(), "Currency"
].unique()

print("Currencies missing FX:")
print(unique_missing)


Missing FX rates: 0
Currencies missing FX:
[]


In [22]:
print("Zero transactions:", (transactions_fx["Transaction"] == 0).sum())
print("Negative transactions:", (transactions_fx["Transaction"] < 0).sum())


Zero transactions: 38
Negative transactions: 3142


In [23]:
print("USD NaN:", transactions_fx["Transaction_USD"].isna().sum())
print("USD min/max:", transactions_fx["Transaction_USD"].min(), transactions_fx["Transaction_USD"].max())


USD NaN: 0
USD min/max: -1597.1558208927863 329947.3677917594


In [24]:
transactions_fx["Transaction_USD"].describe()


count    314754.000000
mean       6742.613783
std       11146.520673
min       -1597.155821
25%         882.676411
50%        2957.167111
75%        7971.041209
max      329947.367792
Name: Transaction_USD, dtype: float64

In [25]:
client_currency_summary = (
    transactions_fx
    .groupby(["Client", "Currency"])
    .agg(
        transaction_count=("Transaction", "count"),
        total_local_amount=("Transaction", "sum"),
        total_usd_amount=("Transaction_USD", "sum")
    )
    .reset_index()
)

client_currency_summary.head()


,Client,Currency,transaction_count,total_local_amount,total_usd_amount
0,11113,AOA,1,329532.049329,646.143147
1,11113,BDT,1,45029.883551,436.748443
2,11113,BND,1,375.905678,272.060272
3,11113,BZD,1,344.660626,172.330313
4,11113,CNY,1,2154.736849,300.718307


In [26]:
client_usd_summary = (
    transactions_fx
    .groupby("Client")
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd_amount=("Transaction_USD", "sum")
    )
    .reset_index()
    .sort_values("total_usd_amount", ascending=False)
)

client_usd_summary.head(10)


,Client,transaction_count,total_usd_amount
5707,BDDDH,18,3.212967e+06
21307,HKKX3,31,2.466483e+06
1122,ABLZb,37,2.148254e+06
2904,AFWb3,25,2.133877e+06
9948,CFHKV,16,2.017376e+06
18819,FWW23,15,1.956643e+06
16100,EJJLb,26,1.945136e+06
17312,FFc11,29,1.925840e+06
3871,AJNNV,25,1.879458e+06
4913,BBGHK,17,1.865101e+06


In [27]:
total_revenue = client_usd_summary["total_usd_amount"].sum()

top_10_revenue = client_usd_summary.head(10)["total_usd_amount"].sum()

print("Total Revenue:", round(total_revenue, 2))
print("Top 10 Revenue:", round(top_10_revenue, 2))
print("Top 10 % of total:", round(top_10_revenue / total_revenue * 100, 2), "%")


Total Revenue: 2122264658.57
Top 10 Revenue: 21551135.58
Top 10 % of total: 1.02 %


In [28]:
client_usd_summary.head(10)


,Client,transaction_count,total_usd_amount
5707,BDDDH,18,3.212967e+06
21307,HKKX3,31,2.466483e+06
1122,ABLZb,37,2.148254e+06
2904,AFWb3,25,2.133877e+06
9948,CFHKV,16,2.017376e+06
18819,FWW23,15,1.956643e+06
16100,EJJLb,26,1.945136e+06
17312,FFc11,29,1.925840e+06
3871,AJNNV,25,1.879458e+06
4913,BBGHK,17,1.865101e+06


In [29]:
client_usd_summary = (
    transactions_fx
    .groupby("Client", as_index=False)
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd_amount=("Transaction_USD", "sum")
    )
    .sort_values("total_usd_amount", ascending=False)
)

top_10 = client_usd_summary.head(10)

print(top_10[["Client", "total_usd_amount"]])
print("Top 10 sum:", top_10["total_usd_amount"].sum())
print("Total revenue:", client_usd_summary["total_usd_amount"].sum())


      Client  total_usd_amount
5707   BDDDH      3.212967e+06
21307  HKKX3      2.466483e+06
1122   ABLZb      2.148254e+06
2904   AFWb3      2.133877e+06
9948   CFHKV      2.017376e+06
18819  FWW23      1.956643e+06
16100  EJJLb      1.945136e+06
17312  FFc11      1.925840e+06
3871   AJNNV      1.879458e+06
4913   BBGHK      1.865101e+06
Top 10 sum: 21551135.579992794
Total revenue: 2122264658.56791


In [30]:
top_10["total_usd_amount"].head()


5707     3.212967e+06
21307    2.466483e+06
1122     2.148254e+06
2904     2.133877e+06
9948     2.017376e+06
Name: total_usd_amount, dtype: float64

In [31]:
top_10["total_usd_amount"].values


array([3212966.77469184, 2466483.30333662, 2148253.64588301,
       2133876.74905923, 2017375.86166505, 1956643.45954788,
       1945136.25398947, 1925839.96618872, 1879458.27434764,
       1865101.29128334])

In [32]:
top_10["total_usd_amount"].iloc[0]


np.float64(3212966.774691836)

In [33]:
total_revenue = client_usd_summary["total_usd_amount"].sum()
top_10_revenue = client_usd_summary.head(10)["total_usd_amount"].sum()

print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Top 10 Revenue: ${top_10_revenue:,.2f}")
print(f"Top 10 % of total: {top_10_revenue/total_revenue*100:,.2f}%")


Total Revenue: $2,122,264,658.57
Top 10 Revenue: $21,551,135.58
Top 10 % of total: 1.02%


In [34]:
# Ensure Client is string
transactions_fx["Client"] = transactions_fx["Client"].astype(str)

# Extract first character
transactions_fx["Market_Section"] = transactions_fx["Client"].str[0]

# Classify numeric vs non-numeric
transactions_fx["Market_Section"] = transactions_fx["Market_Section"].apply(
    lambda x: x if x.isdigit() else "NonNumeric"
)

transactions_fx[["Client", "Market_Section"]].head()


,Client,Market_Section
0,FLVWa,NonNumeric
1,FLVWa,NonNumeric
2,FLVWa,NonNumeric
3,EKKL3,NonNumeric
4,EKKL3,NonNumeric


In [35]:
transactions_fx["Market_Section"].value_counts()


Market_Section
NonNumeric    314656
1                 54
2                 26
3                 18
Name: count, dtype: int64

In [36]:
market_section_summary = (
    transactions_fx
    .groupby("Market_Section")
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd=("Transaction_USD", "sum"),
        avg_transaction_usd=("Transaction_USD", "mean")
    )
    .reset_index()
    .sort_values("total_usd", ascending=False)
)

market_section_summary


,Market_Section,transaction_count,total_usd,avg_transaction_usd
3,NonNumeric,314656,2.121789e+09,6743.201555
1,2,26,3.317920e+05,12761.232501
0,1,54,1.335952e+05,2473.985566
2,3,18,1.044277e+04,580.153849


In [37]:
country_summary = (
    transactions_fx
    .groupby("Country")
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd=("Transaction_USD", "sum"),
        avg_transaction_usd=("Transaction_USD", "mean")
    )
    .reset_index()
    .sort_values("total_usd", ascending=False)
)

country_summary.head(10)


,Country,transaction_count,total_usd,avg_transaction_usd
93,PERU,4620,3.088695e+07,6685.487798
124,TURKEY,2335,1.694128e+07,7255.367570
27,CHINA,2402,1.689828e+07,7035.085556
51,HUNGARY,2293,1.680710e+07,7329.743088
48,HAITI,2322,1.657090e+07,7136.478344
50,HONG KONG,2343,1.650004e+07,7042.272315
15,BOLIVIA,2342,1.647502e+07,7034.595314
69,LIBYAN ARAB JAMAHIRIYA,2364,1.646498e+07,6964.881107
40,EURO,2358,1.644853e+07,6975.625787
21,CAMBODIA,2374,1.641456e+07,6914.303530


In [38]:
total_rev = country_summary["total_usd"].sum()

top_5_rev = country_summary.head(5)["total_usd"].sum()

print(f"Total Revenue: ${total_rev:,.2f}")
print(f"Top 5 Revenue: ${top_5_rev:,.2f}")
print(f"Top 5 % of total: {top_5_rev/total_rev*100:,.2f}%")


Total Revenue: $2,122,264,658.57
Top 5 Revenue: $98,104,516.02
Top 5 % of total: 4.62%


In [40]:
output_path = "output/Report_1_FPA_Analysis.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    
    # 1️⃣ Client USD Totals
    client_usd_summary.to_excel(
        writer, 
        sheet_name="Client_USD_Totals", 
        index=False
    )
    
    # 2️⃣ Client Currency Summary
    client_currency_summary.to_excel(
        writer, 
        sheet_name="Client_Currency_Summary", 
        index=False
    )
    
    # 3️⃣ Country Summary
    country_summary.to_excel(
        writer,
        sheet_name="Country_Summary",
        index=False
    )
    
    # 4️⃣ Full Transaction Detail
    transactions_fx.to_excel(
        writer,
        sheet_name="Detail_Data",
        index=False
    )
    
    # 5️⃣ Summary KPIs sheet
    summary_df = pd.DataFrame({
        "Metric": [
            "Total Revenue (USD)",
            "Total Transactions",
            "Top 10 Client Revenue (USD)",
            "Top 10 % of Total Revenue",
            "Top 5 Countries % of Total Revenue"
        ],
        "Value": [
            total_revenue,
            len(transactions_fx),
            top_10_revenue,
            top_10_revenue/total_revenue,
            top_5_rev/total_rev
        ]
    })
    
    summary_df.to_excel(writer, sheet_name="Summary", index=False)

print("Excel report generated successfully.")


Excel report generated successfully.


In [41]:
os.listdir("output")


['.ipynb_checkpoints', 'file_index.csv', 'Report_1_FPA_Analysis.xlsx']

In [42]:
latam_countries = [
    "ARGENTINA",
    "BOLIVIA",
    "BRAZIL",
    "CHILE",
    "COLOMBIA",
    "COSTA RICA",
    "CUBA",
    "DOMINICAN REPUBLIC",
    "ECUADOR",
    "EL SALVADOR",
    "GUATEMALA",
    "HAITI",
    "HONDURAS",
    "MEXICO",
    "NICARAGUA",
    "PANAMA",
    "PARAGUAY",
    "PERU",
    "URUGUAY",
    "VENEZUELA"
]

# Normalize country text
transactions_fx["Country"] = transactions_fx["Country"].str.upper().str.strip()

latam_transactions = transactions_fx[
    transactions_fx["Country"].isin(latam_countries)
].copy()

print("LatAm Transactions:", len(latam_transactions))
print("LatAm Revenue:", latam_transactions["Transaction_USD"].sum())
latam_transactions["Country"].value_counts()


LatAm Transactions: 43314
LatAm Revenue: 293091116.19689023


Country
PERU                  4620
PARAGUAY              2354
BOLIVIA               2342
COLOMBIA              2328
HAITI                 2322
NICARAGUA             2321
COSTA RICA            2312
PANAMA                2286
GUATEMALA             2279
HONDURAS              2276
ARGENTINA             2270
MEXICO                2261
URUGUAY               2260
EL SALVADOR           2241
BRAZIL                2231
CHILE                 2218
VENEZUELA             2199
DOMINICAN REPUBLIC    2194
Name: count, dtype: int64

In [43]:
# 1) Client USD totals (LatAm only)
latam_client_usd = (
    latam_transactions
    .groupby("Client", as_index=False)
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd_amount=("Transaction_USD", "sum")
    )
    .sort_values("total_usd_amount", ascending=False)
)

# 2) Client x Currency (LatAm only)
latam_client_currency = (
    latam_transactions
    .groupby(["Client", "Currency"], as_index=False)
    .agg(
        transaction_count=("Transaction", "count"),
        total_local_amount=("Transaction", "sum"),
        total_usd_amount=("Transaction_USD", "sum")
    )
    .sort_values(["Client", "Currency"])
)

# 3) Country summary (LatAm only)
latam_country_summary = (
    latam_transactions
    .groupby("Country", as_index=False)
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd=("Transaction_USD", "sum"),
        avg_transaction_usd=("Transaction_USD", "mean")
    )
    .sort_values("total_usd", ascending=False)
)

latam_client_usd.head(10), latam_country_summary.head(10)


(      Client  transaction_count  total_usd_amount
 13560  FLYYb                  3     546993.898429
 11792  EJJLb                  6     476203.111170
 12997  FHLW2                  7     460253.158778
 6079   BNbc1                  8     445573.612255
 4676   BEHV1                  4     416237.237580
 4200   BDDDH                  2     387272.098554
 7700   CHHIb                  6     361376.770344
 808    ABLZb                  7     356962.679906
 2133   AFWb3                  4     355350.336433
 11014  EEac3                  4     353669.618957,
        Country  transaction_count     total_usd  avg_transaction_usd
 15        PERU               4620  3.088695e+07          6685.487798
 9        HAITI               2322  1.657090e+07          7136.478344
 1      BOLIVIA               2342  1.647502e+07          7034.595314
 14    PARAGUAY               2354  1.629506e+07          6922.283660
 8    GUATEMALA               2279  1.607883e+07          7055.213481
 12   NICARAGUA   

In [44]:
latam_output_path = "output/Report_2_LatAm_Leaders.xlsx"

with pd.ExcelWriter(latam_output_path, engine="openpyxl") as writer:
    latam_client_usd.to_excel(writer, sheet_name="Client_USD_Totals", index=False)
    latam_client_currency.to_excel(writer, sheet_name="Client_Currency_Summary", index=False)
    latam_country_summary.to_excel(writer, sheet_name="Country_Summary", index=False)
    latam_transactions.to_excel(writer, sheet_name="Detail_Data", index=False)

print("Created:", latam_output_path)


Created: output/Report_2_LatAm_Leaders.xlsx


In [45]:
import os
os.listdir("data/Transactions/Transacciones")


['2024']

In [46]:
risk_countries = ["RUSSIA", "CUBA", "CHINA", "VENEZUELA"]

# Ensure country is clean
transactions_fx["Country"] = transactions_fx["Country"].astype(str).str.upper().str.strip()

# Q4 2024 filter (Oct-Dec 2024)
q4_2024 = transactions_fx[
    (transactions_fx["year"] == 2024) &
    (transactions_fx["statement_date"].dt.quarter == 4)
]

q4_2024_risk = q4_2024[q4_2024["Country"].isin(risk_countries)].copy()

risk_clients_q4_2024 = (
    q4_2024_risk[["Client"]]
    .drop_duplicates()
    .sort_values("Client")
)

print("Q4 2024 transactions:", len(q4_2024))
print("Q4 2024 risk-country transactions:", len(q4_2024_risk))
print("Unique clients:", len(risk_clients_q4_2024))

q4_2024_risk["Country"].value_counts()


Q4 2024 transactions: 102206
Q4 2024 risk-country transactions: 1501
Unique clients: 1355


Country
CHINA        771
VENEZUELA    730
Name: count, dtype: int64

In [47]:
risk_revenue_q4_2024 = q4_2024_risk["Transaction_USD"].sum()
total_revenue_q4_2024 = q4_2024["Transaction_USD"].sum()

print(f"Q4 2024 Total Revenue: ${total_revenue_q4_2024:,.2f}")
print(f"Q4 2024 Risk Revenue: ${risk_revenue_q4_2024:,.2f}")
print(f"Risk % of Q4 Revenue: {risk_revenue_q4_2024 / total_revenue_q4_2024 * 100:.2f}%")


Q4 2024 Total Revenue: $662,249,175.68
Q4 2024 Risk Revenue: $9,886,486.56
Risk % of Q4 Revenue: 1.49%


In [48]:
# --- Risk Country Summary (Q4 2024) ---

risk_summary_q4 = (
    q4_2024_risk
    .groupby("Country", as_index=False)
    .agg(
        transaction_count=("Transaction", "count"),
        total_usd=("Transaction_USD", "sum")
    )
    .sort_values("total_usd", ascending=False)
)

# Unique client list
risk_clients_list = (
    q4_2024_risk[["Client"]]
    .drop_duplicates()
    .sort_values("Client")
)

risk_summary_q4, risk_clients_list.head()


(     Country  transaction_count     total_usd
 0      CHINA                771  5.531656e+06
 1  VENEZUELA                730  4.354830e+06,
        Client
 268674  AAAAV
 228700  AAABM
 213887  AAACM
 237700  AAAFZ
 219867  AAAKM)

In [49]:
task2_output_path = "output/Task_2_LatAm_and_Risk.xlsx"

with pd.ExcelWriter(task2_output_path, engine="openpyxl") as writer:
    
    # --- LATAM SECTION ---
    latam_client_usd.to_excel(writer, sheet_name="LatAm_Client_USD", index=False)
    latam_client_currency.to_excel(writer, sheet_name="LatAm_Client_Currency", index=False)
    latam_country_summary.to_excel(writer, sheet_name="LatAm_Country_Summary", index=False)
    latam_transactions.to_excel(writer, sheet_name="LatAm_Detail", index=False)
    
    # --- RISK SECTION ---
    risk_summary_q4.to_excel(writer, sheet_name="Q4_2024_Risk_Summary", index=False)
    risk_clients_list.to_excel(writer, sheet_name="Q4_2024_Risk_Clients", index=False)
    q4_2024_risk.to_excel(writer, sheet_name="Q4_2024_Risk_Detail", index=False)

print("Created:", task2_output_path)


Created: output/Task_2_LatAm_and_Risk.xlsx


In [50]:
python app_report1.py


SyntaxError: invalid syntax (3116599182.py, line 1)

In [51]:
base_dir = r"data/Transactions/Transacciones"  # adjust if needed
file_index = build_file_index(base_dir)

print("Files found:", len(file_index))
file_index.head()


Files found: 537


,filepath,filetype,year,month_folder,statement_folder,statement_date,month,quarter,source_id
0,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,04-01-24,2024-01-04,1,1,2024-01-04_000
1,data/Transactions/Transacciones\2024\01 Januar...,xlsx,2024,01 January,04-01-24,2024-01-04,1,1,2024-01-04_001
2,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,11-01-24,2024-01-11,1,1,2024-01-11_000
3,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,11-01-24,2024-01-11,1,1,2024-01-11_001
4,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,11-01-24,2024-01-11,1,1,2024-01-11_002


In [52]:
base_dir = r"data/Transactions/Transacciones"  # adjust if needed
file_index = build_file_index(base_dir)

print("Files found:", len(file_index))
file_index.head()


Files found: 537


,filepath,filetype,year,month_folder,statement_folder,statement_date,month,quarter,source_id
0,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,04-01-24,2024-01-04,1,1,2024-01-04_000
1,data/Transactions/Transacciones\2024\01 Januar...,xlsx,2024,01 January,04-01-24,2024-01-04,1,1,2024-01-04_001
2,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,11-01-24,2024-01-11,1,1,2024-01-11_000
3,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,11-01-24,2024-01-11,1,1,2024-01-11_001
4,data/Transactions/Transacciones\2024\01 Januar...,csv,2024,01 January,11-01-24,2024-01-11,1,1,2024-01-11_002


In [53]:
base_dir = r"data/Transactions/Transacciones"  # adjust if needed
file_index = build_file_index(base_dir)

print("Files found:", len(file_index))
file_index.head()

available_years = sorted(file_index["year"].dropna().unique())
available_years


Files found: 537


[np.int64(2024)]